In [ ]:
import requests
import pandas as pd

In [29]:
r = requests.get('https://www.afl.com.au/news/564/afl-injury-list')
dfs = pd.read_html(r.text)

In [30]:
df = pd.DataFrame()
for df_part in dfs:
    updated_text = df_part.iloc[-1,0]
    df_part = df_part.iloc[:-1]
    date = updated_text[updated_text.find(':')+2:]
    df_part['Updated'] = date
    df = df.append(df_part)
df.to_csv('injuries.csv')

<ipython-input-30-3498eece0330>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_part['Updated'] = date
